In [73]:

#!/usr/bin/env python3

import argparse
import json
import os
import importlib.util

from src.learner.raytune_learner import TuneWrapper as StimulusTuneWrapper
from launch_utils import import_class_from_file, get_experiment
from src.utils.yaml_model_schema import YamlRayConfigLoader
import torch 
import src.data.handlertorch as handlertorch
from src.learner.predict import PredictWrapper


model_path = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus/stimulus/fpkm_model/fpkm_dummy_model.py"
json_experiment = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus/stimulus/fpkm_model/fpkm_pipelinegenerated.json"
config_path = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus/stimulus/fpkm_model/fpkm_dummy_model.yaml"
data_path = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus_old/stimulus/work/9d/0ebf1c448bbe1a0ca653b26bea66f7/test-fpkm-#2-trans.csv"
best_config_path = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus/stimulus/fpkm_model/best_config.json"






# import the model correctly but do not initialize it yet, ray_tune does that itself
model_class = import_class_from_file(model_path)
model_class()
# read json and retrieve experiment name and then initialize the experiment class
experiment_name = None
with open(json_experiment, 'r') as in_json:
    d = json.load(in_json)
    experiment_name = d["experiment"]
experiment = get_experiment(experiment_name)


dataset = handlertorch.TorchDataset(os.path.abspath(data_path), experiment, split=None)
train_data = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)



# Create the learner
#learner = StimulusTuneWrapper(config_path, model_class, data_path, experiment)


# Tune the model
# learner.tune()
# save best config
#learner.store_best_config(best_config_path)



In [74]:
# read in config
import torch.nn as nn

config_loader = YamlRayConfigLoader(config_path)
config = config_loader.config

loss_dict = {"loss_fn": "PoissonNLLLoss"}
for key, loss_fn in loss_dict.items():
    try:
        loss_dict[key] = getattr(nn, loss_fn)()
    except AttributeError:
        raise ValueError(f"Invalid loss function: {loss_fn}, check PyTorch for documentation on available loss functions")


In [75]:
loss_dict

{'loss_fn': PoissonNLLLoss()}

In [76]:
wrapper = PredictWrapper(model_class(), data_path, experiment, loss_dict, split = 1, batch_size = 1)

In [79]:
wrapper.compute_metric("loss")

{'fpkm': tensor([[1.]])}
{'fpkm': tensor([[1.]])}
{'fpkm': tensor([[1.]])}
{'fpkm': tensor([[1.]])}
{'fpkm': tensor([[1.]])}
{'fpkm': tensor([[1.]])}
{'fpkm': tensor([[1.]])}


2.6470473316249303